In [1]:
import os
import numpy as np
import pandas as pd
import eipy.ei as e
import tensorflow as tf
from importlib import reload
from sklearn.model_selection import train_test_split
from sklearn.impute import KNNImputer
from sklearn.metrics import accuracy_score

from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
import pandas as pd
from eipy.additional_ensembles import MeanAggregation, CES
from eipy.metrics import fmax_score
from sklearn.metrics import roc_auc_score, matthews_corrcoef

from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Input, Concatenate, LSTM

2024-02-01 18:51:25.200664: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-01 18:51:25.228176: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-01 18:51:25.228201: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-01 18:51:25.228972: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-01 18:51:25.233610: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-01 18:51:25.234569: I tensorflow/core/platform/cpu_feature_guard.cc:1

data

In [2]:
directory = "/home/opc/eipy/eipy/COVID-19 data"
data={}
for file_name in os.listdir(directory):
    if file_name.endswith('.csv'):
        key = file_name.split('.')[0]
        file_path = os.path.join(directory, file_name)
        df = pd.read_csv(file_path)
        df = df.drop(columns=["NEW_MASKED_MRN"])
        imputer = KNNImputer(n_neighbors=5)
        df = imputer.fit_transform(df)
        data[key] = df

In [3]:
labels = pd.read_csv('/home/opc/eipy/eipy/COVID-19 data/Basics/outcome.csv')
labels = labels["DECEASED_INDICATOR"]

In [4]:
X_train = {}
X_test = {}
for k, v in data.items():
    X_train[k], X_test[k], y_train, y_test = train_test_split(v, labels, test_size=0.2, stratify=labels)

In [5]:
for k,v in X_train.items():
    print(k, v.shape)

admission (3826, 23)
comorbidities (3826, 22)
vitals (3826, 9)
xgboost (3826, 98)
labs (3826, 44)


dl bps

In [6]:
input_dim = 98

# Architecture 1: Simple Feedforward Neural Network
model_1 = Sequential()
model_1.add(Dense(64, input_dim=input_dim, activation='relu'))
model_1.add(Dense(32, activation='relu'))
model_1.add(Dense(1, activation='sigmoid'))
model_1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Architecture 2: Deep Feedforward Neural Network with Dropout
model_2 = Sequential()
model_2.add(Dense(128, input_dim=input_dim, activation='relu'))
model_2.add(Dropout(0.5))
model_2.add(Dense(64, activation='relu'))
model_2.add(Dropout(0.3))
model_2.add(Dense(32, activation='relu'))
model_2.add(Dense(1, activation='sigmoid'))
model_2.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Architecture 3: Wide and Deep Neural Network
input_layer = Input(shape=(input_dim,))
wide_branch = Dense(32, activation='relu')(input_layer)
deep_branch = Dense(64, activation='relu')(input_layer)
deep_branch = Dense(32, activation='relu')(deep_branch)
merged = Concatenate()([wide_branch, deep_branch])
output_layer = Dense(1, activation='sigmoid')(merged)
model_3 = Model(inputs=input_layer, outputs=output_layer)
model_3.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Architecture 4: Deep Neural Network without Sequential Structure
model_4 = Sequential()
model_4.add(Dense(128, input_dim=input_dim, activation='relu'))
model_4.add(Dense(64, activation='relu'))
model_4.add(Dense(32, activation='relu'))
model_4.add(Dense(1, activation='sigmoid'))
model_4.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

d_base_predictors = {"simple" : model_1,
                     "deep_w_dropout" : model_2,
                     "wide_n_deep" : model_3,
                     "deeper" : model_4}

In [ ]:
base_predictors = {
                    'ADAB': AdaBoostClassifier(),
                    'XGB': XGBClassifier(),
                    'DT': DecisionTreeClassifier(),
                    'RF': RandomForestClassifier(),
                    'GB': GradientBoostingClassifier(),
                    'KNN': KNeighborsClassifier(),
                    'LR': LogisticRegression(),
                    'NB': GaussianNB(),
                    'MLP': MLPClassifier(),
                    'SVM': SVC(probability=True),
}

In [64]:
ensemble_predictors = {
                    'Mean' : MeanAggregation(),
                    'CES' : CES(scoring=lambda y_test, y_pred: fmax_score(y_test, y_pred)[0]),
                    'S.ADAB': AdaBoostClassifier(),
                    'S.XGB': XGBClassifier(),
                    'S.DT': DecisionTreeClassifier(),
                    "S.RF": RandomForestClassifier(),
                    'S.GB': GradientBoostingClassifier(),
                    'S.KNN': KNeighborsClassifier(),
                    'S.LR': LogisticRegression(),
                    'S.NB': GaussianNB(),
                    'S.MLP': MLPClassifier(),
                    'S.SVM': SVC(probability=True),
}

In [8]:
metrics = {
            'f_max': fmax_score,
            'auc': roc_auc_score,
            'mcc': matthews_corrcoef
            }

EI

In [54]:
####HI OVER HERE###
reload(e)
###HEYOOOOOO#

<module 'eipy.ei' from '/home/opc/eipy/eipy/ei.py'>

In [55]:
EI = e.EnsembleIntegration(
                        k_outer=5,
                        k_inner=5,
                        n_samples=1,
                        sampling_strategy="undersampling",
                        sampling_aggregation=None,
                        n_jobs=-1,
                        metrics=metrics,
                        random_state=38,
                        project_name="diabetes",
                        model_building=True,
                        )

In [56]:
for k,v in X_train.items():
    if k == "xgboost":
        EI.fit_base(v, y_train, modality_name=k, base_predictors=d_base_predictors)
    else:
        EI.fit_base(v, y_train, modality_name=k, base_predictors=base_predictors)

Training base predictors on admission...
        
... for ensemble performance analysis...


Generating ensemble training data: |          |  0%2024-02-01 19:38:39.950006: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-01 19:38:39.979542: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-01 19:38:39.979588: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-01 19:38:39.980328: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-01 19:38:39.985098: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-01 19:38:39.985264


... for final ensemble...


Generating ensemble training data: |██████████|100%
Training final base predictors: |██████████|100%




Training base predictors on comorbidities...
        
... for ensemble performance analysis...


...

Generating final ensemble training data: |██████████|100%




Training base predictors on labs...
        
... for ensemble performance analysis...


Generating ensemble training data: |██████████|100%
Generating ensemble test data: |██████████|100%



... for final ensemble...


Generating ensemble training data: |██████████|100%
Training final base predictors: |██████████|100%


In [57]:
EI.base_summary["metrics"]

modality       admission                                                    \
base predictor      ADAB        DT        GB       KNN        LR       MLP   
f_max           0.433340  0.433251  0.433340  0.433251  0.433518  0.433457   
auc             0.477557  0.488118  0.493628  0.478130  0.515388  0.497173   
mcc            -0.028124 -0.021260 -0.014054 -0.035814  0.013157 -0.022633   

modality                                                ...    vitals  \
base predictor        NB        RF       SVM       XGB  ...        LR   
f_max           0.433251  0.433429  0.433518  0.433607  ...  0.433994   
auc             0.500499  0.475466  0.501901  0.486902  ...  0.510698   
mcc            -0.000943 -0.028557  0.003662 -0.013551  ...  0.026355   

modality                                                          \
base predictor       MLP        NB        RF       SVM       XGB   
f_max           0.435216  0.434051  0.433532  0.433429  0.433854   
auc             0.514472  0.488541  0.511194  0.504417  0.501730   
mcc             0.013675 -0.013345  0.024788  0.016558  0.005822   

modality              xgboost                                  
base predictor deep_w_dropout    deeper    simple wide_n_deep  
f_max                0.433670  0.434449  0.433251    0.433997  
auc                  0.492642  0.502472  0.498601    0.503209  
mcc                 -0.014263 -0.003718 -0.004049    0.020478  

[3 rows x 44 columns]

In [65]:
EI.fit_ensemble(ensemble_predictors=ensemble_predictors)

Analyzing ensembles: |██████████|100%
Training final ensemble models: |██████████|100%


In [66]:
EI.ensemble_summary["metrics"]

,Mean,CES,S.ADAB,S.XGB,S.DT,S.RF,S.GB,S.KNN,S.LR,S.NB,S.MLP,S.SVM
f_max,0.539754,0.587213,0.579256,0.586733,0.439114,0.596283,0.593645,0.565725,0.602182,0.593845,0.598208,0.593131
auc,0.734981,0.783556,0.777737,0.780159,0.610430,0.799361,0.798133,0.741650,0.809088,0.790818,0.796776,0.771641
mcc,0.330563,0.391161,0.359250,0.329216,0.217678,0.389841,0.378542,0.322176,0.413033,0.406840,0.379183,0.391253


In [67]:
preferred_ensemble_key = EI.ensemble_summary["metrics"].loc["auc"].idxmax()
y_pred = EI.predict(X_dict=X_test, ensemble_model_key=preferred_ensemble_key)
roc_auc_score(y_test, y_pred)

30/30 [==============================] - 0s 737us/step


0.8008834114952559